# Kế hoạch chạy private test
**Tiền xử lý dữ liệu**
1.  Chạy dataset Process Chunking để xử lý

Link notebook : https://www.kaggle.com/code/nguyendinhhaduong/hcmus-foursight-processdata-for-private-test/edit

2. Chạy AI-mon lab để khả năng nhỏ giữa các model để đổi nhãn

Link notebook : https://www.kaggle.com/code/nguyendinhhaduong/hcmus-foursight-aimon-s-hallucinatdetectioneval/edit

**Xử lý các model **


1. Chạy model đạt kết quả 801

2. Chạy model đạt kết quả 807

3. Chạy model đạt kết quả 799

Xong kết hợp 3 model lại với nhau 

Sau khi kết hợp tìm các giá trị khác nhau giải ra số câu đúng câu sai xong dựa ground truth của AI -mon lab đổi nhãn 

Chỉ đổi trong các giá trị khác nhau 

- Ưu tiên chiến thuật 0.8 -1.0 nếu đang là no trong các giá trị khác nhau thì đổi nhãn 1 trong hai giá trị {extrinsic , intrinsic) (Một giá trị trong hai bảng thấp hơn )

- Nếu đang là intrinsic, extrinsic có ground truth 0.1 thì đổi thành no
   


**Cần code Hàm** 

1. Tạo file hyphonthies, permise của private test 

2. Code hàm dự đoán kết quả của một model khi nhập tên

3. Code hàm check model dự đoán do huấn luyện

799:  https://www.kaggle.com/code/nguyendinhhaduong/nli-crossencoder

801: https://huggingface.co/Nguyenhhh/vihallu_model_mdeberta_nli_hard30

807: https://huggingface.co/Nguyenhhh/vihallu_model_mdeberta_nli_hard30/tree/main

4. Code hàm check giữa hai đáp án df. In các giá trị khác nhau + ground truth

5. So sánh các giá trị khác nhau giữa các bảng. Các id có trùng không 

6. Check file submition đúng form và thống kê nhãn. 


# Thư viện cần thiết 

In [1]:
import pandas as pd

## 1. 1 . Tạo file hyphonthies, permise của private test 

In [2]:
def process_csv(input_path, output_path):
    df = pd.read_csv(input_path)
    if "Unnamed: 0" in df.columns:
        df = df.drop(columns=["Unnamed: 0"])
    before = len(df)
    df = df.dropna(subset=["context", "prompt", "response"])
    after = len(df)
    removed = before - after
    if removed > 0:
        print(f"⚠️ Đã xóa {removed} dòng bị trống trong context, prompt hoặc response")
    df["premise"] = df["prompt"].astype(str) + " [SEP] " + df["context"].astype(str)
    df["hypothesis"] = df["response"].astype(str)
    df.to_csv(output_path, index=False)
    print(f"✅ Đã xử lý và lưu file tại: {output_path}")

    return df

In [3]:
processed_df = process_csv("/kaggle/input/public-test-process/test.csv", "public_processed.csv")

✅ Đã xử lý và lưu file tại: public_processed.csv


In [4]:
processed_df = process_csv("/kaggle/input/hcmus-foursight-train-set-process/vihallu_train.csv", "train_processed.csv")

✅ Đã xử lý và lưu file tại: train_processed.csv


In [23]:
processed_df = process_csv("/kaggle/input/private-test-process/Private-test Context_new.csv", "private_processed.csv")

✅ Đã xử lý và lưu file tại: private_processed.csv


In [4]:
df_test=  pd.read_csv("/kaggle/working/public_processed.csv")
df_test

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,id,context,prompt,response,label,premise,hypothesis
0,b709059b-b3b6-4ac2-bb88-2c794e2cc219,"Putin ngày 14 tháng 10 năm 2009, đưa ra đề ngh...",Ý nhĩa cũa viẹc tổ chưc cuôc thi Intervision l...,"Cuộc thi Intervision, dự kiến lấy cảm hứng từ ...",NaN,Ý nhĩa cũa viẹc tổ chưc cuôc thi Intervision l...,"Cuộc thi Intervision, dự kiến lấy cảm hứng từ ..."
1,7dc35ef5-c4b7-4538-ab90-627b9cbd896e,"Thông qua những quyết nghị này, Viện nguyên lã...",Quyền lực của Viện nguyên lão đã giảm mạnh khi...,"Sai. Thực tế, Viện nguyên lão vẫn duy trì quyề...",NaN,Quyền lực của Viện nguyên lão đã giảm mạnh khi...,"Sai. Thực tế, Viện nguyên lão vẫn duy trì quyề..."
2,cfdfa010-f61c-4845-91c9-23f79be2b88b,Nơi cư ngụ truyền thống Mông Cổ được gọi là mộ...,Theo quan điểm của ai thì kiến trúc truyền thố...,"Ngoài N. Chultem, một số học giả như nhà sử họ...",NaN,Theo quan điểm của ai thì kiến trúc truyền thố...,"Ngoài N. Chultem, một số học giả như nhà sử họ..."
3,31b33c97-2f59-4e72-8707-f47de204d7f9,Là thành phố thủ đô và có vị trí ở khu vực tru...,Hà Nội hiện nay có tuyến đường sắt quốc tế kết...,Hà Nội hiện nay có tuyến đường sắt quốc tế trự...,NaN,Hà Nội hiện nay có tuyến đường sắt quốc tế kết...,Hà Nội hiện nay có tuyến đường sắt quốc tế trự...
4,a2c83a00-e8b7-4236-86ce-5e0104df074a,Từ 1974 đến nay tốc độ phát triển tuy chậm lại...,Câu hỏi gài bẫy: Tiền đề để trở thành một quốc...,Tiền đề để Nhật Bản trở thành quốc gia cho vay...,NaN,Câu hỏi gài bẫy: Tiền đề để trở thành một quốc...,Tiền đề để Nhật Bản trở thành quốc gia cho vay...
...,...,...,...,...,...,...,...
995,3e6360c4-3f4a-4cc0-9cb8-aa88d5aa73ab,Thời đại đồ đồng là một phần trong hệ thống ba...,Các nền văn minh trên khu vực ven sông chủ yếu...,Các nền văn minh ven sông bắt đầu phát triển m...,NaN,Các nền văn minh trên khu vực ven sông chủ yếu...,Các nền văn minh ven sông bắt đầu phát triển m...
996,a7ce3e0a-9f40-4284-a847-3e395c6d2d99,Sức khỏe của Elizabeth vẫn tốt cho đến mùa thu...,Sự kiện nà đã làm cho Elizabeth sôc vào thang ...,Elizabeth bị sốc vào tháng 2 năm 1603 do sự ra...,NaN,Sự kiện nà đã làm cho Elizabeth sôc vào thang ...,Elizabeth bị sốc vào tháng 2 năm 1603 do sự ra...
997,34a351e9-da10-4a15-b370-2412d9e95fcc,Hải ly được biết là có xây những đập rất lớn. ...,Đập lớn nhất của hải ly được tìm thấy ở Montan...,Đập lớn nhất của hải ly được tìm thấy ở Montan...,NaN,Đập lớn nhất của hải ly được tìm thấy ở Montan...,Đập lớn nhất của hải ly được tìm thấy ở Montan...
998,be16b564-c610-462b-98b3-6bb9980b7433,Nhật Bản còn có một mỹ danh trong cộng đồng ng...,Ý nghĩa hoa anh đào là gì?,"Hoa anh đào biểu thị cho sự nhạy cảm, yêu cái ...",NaN,Ý nghĩa hoa anh đào là gì? [SEP] Nhật Bản còn ...,"Hoa anh đào biểu thị cho sự nhạy cảm, yêu cái ..."


##  2.1  Sinh file submit cho model 

### 2.1.1 Model 799 ( load từ kaggle )

In [6]:
save_path = "/kaggle/input/nli-crossencoder-593e52/best_nli_sota_model.pth"

In [11]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, DebertaV2ForSequenceClassification

# ===============================
# 1. Config
# ===============================
class Config:
    TEST_PATH = "/kaggle/working/public_processed.csv"
    MODEL_NAME = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"  # backbone
    WORKDIR = "/kaggle/input/nli-crossencoder-dontlekge"             # chứa checkpoint .pth
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    MAX_LEN = 1024
    BATCH_SIZE = 16
test_df = pd.read_csv(Config.TEST_PATH)

# ===============================
# 2. Dataset cho inference
# ===============================
tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_NAME)
test_df = pd.read_csv(Config.TEST_PATH)

class NLIDataset(Dataset):
    def __init__(self, df):
        self.premises = df['premise'].astype(str).tolist()
        self.hypotheses = df['hypothesis'].astype(str).tolist()
    def __len__(self): return len(self.premises)
    def __getitem__(self, idx):
        item = tokenizer(
            self.premises[idx], self.hypotheses[idx],
            padding="max_length", truncation=True,
            max_length=Config.MAX_LEN, return_tensors="pt"
        )
        return {k: v.squeeze(0) for k, v in item.items()}

test_loader = DataLoader(NLIDataset(test_df), batch_size=Config.BATCH_SIZE, shuffle=False)

# ===============================
# 3. Load model + checkpoint
# ===============================
class HallucinationNLIModel(DebertaV2ForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)

# khởi tạo model
model = HallucinationNLIModel.from_pretrained(Config.MODEL_NAME).to(Config.DEVICE)

# load state_dict đã train
ckpt_path = "/kaggle/input/nli-crossencoder/best_nli_sota_model.pth"
state_dict = torch.load(ckpt_path, map_location=Config.DEVICE)
model.load_state_dict(state_dict)
model.eval()

# ===============================
# 4. Inference
# ===============================
test_preds = []
with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(Config.DEVICE) for k, v in batch.items()}
        logits = model(**batch).logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        test_preds.extend(preds)

# map lại nhãn
id2label = {0: "no", 1: "extrinsic", 2: "intrinsic"}
final_labels = [id2label[p] for p in test_preds]

# ===============================
# 5. Xuất submission
# ===============================
submission_df = pd.DataFrame({
    "id": test_df["id"],
    "predict_label": final_labels
})
submission_df.to_csv("submission_nli_sota799.csv", index=False)
print(submission_df.head())



                                     id predict_label
0  b709059b-b3b6-4ac2-bb88-2c794e2cc219     extrinsic
1  7dc35ef5-c4b7-4538-ab90-627b9cbd896e     extrinsic
2  cfdfa010-f61c-4845-91c9-23f79be2b88b     extrinsic
3  31b33c97-2f59-4e72-8707-f47de204d7f9     intrinsic
4  a2c83a00-e8b7-4236-86ce-5e0104df074a     intrinsic


>  Tí thử với file của mình lỡ khi cao hơn 

In [1]:
import pandas as pd

def compare_predict_label(df1: pd.DataFrame, df2: pd.DataFrame, col: str = "predict_label"):
    """
    So sánh giá trị của cột `col` giữa 2 DataFrame cùng index.
    Trả về số lượng khác nhau và DataFrame chứa các dòng khác nhau.
    """
    # Mask dòng khác nhau
    diff_mask = df1[col] != df2[col]

    # Đếm số lượng
    num_diff = diff_mask.sum()

    # Ghép 2 df để xem khác biệt
    diff_rows = pd.concat([df1[diff_mask], df2[diff_mask]], axis=1, keys=["df1", "df2"])

    print(f"🔎 Số giá trị khác nhau ở cột `{col}`: {num_diff}")
    return num_diff, diff_rows




In [2]:
df1 = pd.read_csv("/kaggle/input/799privatetest/submit.csv")
df2 = pd.read_csv("/kaggle/input/800privatetest/submit.csv")

num_diff, diff_rows1 = compare_predict_label(df1, df2)

print(diff_rows1.head())  # in vài dòng đầu khác nhau

🔎 Số giá trị khác nhau ở cột `predict_label`: 113
                                     df1                \
                                      id predict_label   
2   cd056e1b-51f6-4adf-939c-e742645437bf            no   
13  b92d4a02-6d28-45f3-bd74-ad791ca812ad     extrinsic   
17  37f0e8cd-c00d-4b30-9c6e-0fc1931afc61            no   
39  16ad0c8c-218a-4b77-975f-81be21d61460     intrinsic   
54  018cc4f2-02fa-4fc2-9c60-f6d603113679     extrinsic   

                                     df2                
                                      id predict_label  
2   cd056e1b-51f6-4adf-939c-e742645437bf     intrinsic  
13  b92d4a02-6d28-45f3-bd74-ad791ca812ad            no  
17  37f0e8cd-c00d-4b30-9c6e-0fc1931afc61     intrinsic  
39  16ad0c8c-218a-4b77-975f-81be21d61460            no  
54  018cc4f2-02fa-4fc2-9c60-f6d603113679     intrinsic  


In [6]:
import pandas as pd

# Đọc dữ liệu
df1 = pd.read_csv("/kaggle/input/799privatetest/submit.csv")
df2 = pd.read_csv("/kaggle/input/800privatetest/submit.csv")
df3 = pd.read_csv("/kaggle/input/nli-crossencoder-593e52/submission_nli_sota.csv")

# Gom các predict_label thành 1 DataFrame
votes = pd.DataFrame({
    "df1": df1["predict_label"],
    "df2": df2["predict_label"],
    "df3": df3["predict_label"],
})

# Áp dụng majority voting
def majority_vote(row):
    return row.value_counts().idxmax()

df_final = df1.copy()
df_final["predict_label"] = votes.apply(majority_vote, axis=1)

# Lưu file kết quả
df_final.to_csv("submission_majority.csv", index=False)

print("✅ Done! File lưu tại submission_majority.csv")


✅ Done! File lưu tại submission_majority.csv


In [7]:
df1 = pd.read_csv("/kaggle/working/submission_majority.csv")
df2 = pd.read_csv("/kaggle/input/800privatetest/submit.csv")

num_diff, diff_rows2 = compare_predict_label(df1, df2)

print(diff_rows2.head())  # in vài dòng đầu khác nhau

🔎 Số giá trị khác nhau ở cột `predict_label`: 82
                                      df1                \
                                       id predict_label   
2    cd056e1b-51f6-4adf-939c-e742645437bf            no   
13   b92d4a02-6d28-45f3-bd74-ad791ca812ad     extrinsic   
17   37f0e8cd-c00d-4b30-9c6e-0fc1931afc61            no   
54   018cc4f2-02fa-4fc2-9c60-f6d603113679     extrinsic   
100  ecc2ea60-7904-47ad-b38d-cc195ebdcd0f     extrinsic   

                                      df2                
                                       id predict_label  
2    cd056e1b-51f6-4adf-939c-e742645437bf     intrinsic  
13   b92d4a02-6d28-45f3-bd74-ad791ca812ad            no  
17   37f0e8cd-c00d-4b30-9c6e-0fc1931afc61     intrinsic  
54   018cc4f2-02fa-4fc2-9c60-f6d603113679     intrinsic  
100  ecc2ea60-7904-47ad-b38d-cc195ebdcd0f     intrinsic  


In [4]:
diff_rows2

df1                \
                                        id predict_label   
2     cd056e1b-51f6-4adf-939c-e742645437bf            no   
11    38512c11-3053-41b1-85e6-c71d30bfb052     extrinsic   
13    b92d4a02-6d28-45f3-bd74-ad791ca812ad     extrinsic   
17    37f0e8cd-c00d-4b30-9c6e-0fc1931afc61            no   
51    6da9f1e6-8073-40f3-956f-01697939c5bc     intrinsic   
...                                    ...           ...   
1924  2118a63e-5d2c-4133-aa7d-0e566ce715f2     extrinsic   
1937  1dcb2e5f-6351-43b7-ac95-991be4441591     extrinsic   
1970  fd9f459a-1c61-4067-bc17-e10cda527c39     extrinsic   
1975  1251ffd1-2768-4225-9afe-bed2db457325            no   
1999  e97126ff-84cc-4178-bdde-f81d9aa09ddb            no   

                                       df2                
                                        id predict_label  
2     cd056e1b-51f6-4adf-939c-e742645437bf     intrinsic  
11    38512c11-3053-41b1-85e6-c71d30bfb052     intrinsic  
13    b92d4a02-6d28-45f3-bd74-ad791ca812ad            no  
17    37f0e8cd-c00d-4b30-9c6e-0fc1931afc61     intrinsic  
51    6da9f1e6-8073-40f3-956f-01697939c5bc            no  
...                                    ...           ...  
1924  2118a63e-5d2c-4133-aa7d-0e566ce715f2     intrinsic  
1937  1dcb2e5f-6351-43b7-ac95-991be4441591     intrinsic  
1970  fd9f459a-1c61-4067-bc17-e10cda527c39            no  
1975  1251ffd1-2768-4225-9afe-bed2db457325     intrinsic  
1999  e97126ff-84cc-4178-bdde-f81d9aa09ddb     intrinsic  

[135 rows x 4 columns]

In [5]:
# Lấy ID từ mỗi bảng
ids1 = set(diff_rows1[('df1', 'id')])
ids2 = set(diff_rows2[('df1', 'id')])

# Tìm giao giữa 2 tập
common_ids = ids1.intersection(ids2)

print(f"Số ID chung: {len(common_ids)}")
print(list(common_ids)[:10])



Số ID chung: 82
['8ec107e4-557f-4c39-bf90-643e36dc2a8f', '1825d5cc-b67c-4664-8543-467f558bb749', 'e3c0e21b-39b4-4c1b-94c4-fb531e68581e', '362ecdeb-d15d-4e15-ba48-c0e366ab7f0f', 'c7a64990-1dcf-40bf-b463-94a7be5cc1b9', '70d2c7d0-5e2c-462f-9893-2159517060f1', 'cd811169-bc57-4d01-b35b-1233716fcbfe', 'a521df87-f717-4d22-a53e-794e24d6ec29', '4fc2b06e-6cc2-4f81-9f49-74dddcc689a3', 'c6560e86-abc3-4f47-abc9-a094ca50362b']


In [32]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, DebertaV2ForSequenceClassification

# ===============================
# 1. Config
# ===============================
class Config:
    TEST_PATH = "/kaggle/working/private_processed.csv"
    MODEL_NAME = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"  # backbone
    WORKDIR = "/kaggle/input/nli-crossencoder-dontlekge"             # chứa checkpoint .pth
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    MAX_LEN = 512
    BATCH_SIZE = 16
test_df = pd.read_csv(Config.TEST_PATH)

# ===============================
# 2. Dataset cho inference
# ===============================
tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_NAME)
test_df = pd.read_csv(Config.TEST_PATH)

class NLIDataset(Dataset):
    def __init__(self, df):
        self.premises = df['premise'].astype(str).tolist()
        self.hypotheses = df['hypothesis'].astype(str).tolist()
    def __len__(self): return len(self.premises)
    def __getitem__(self, idx):
        item = tokenizer(
            self.premises[idx], self.hypotheses[idx],
            padding="max_length", truncation=True,
            max_length=Config.MAX_LEN, return_tensors="pt"
        )
        return {k: v.squeeze(0) for k, v in item.items()}

test_loader = DataLoader(NLIDataset(test_df), batch_size=Config.BATCH_SIZE, shuffle=False)

# ===============================
# 3. Load model + checkpoint
# ===============================
class HallucinationNLIModel(DebertaV2ForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)

# khởi tạo model
model = HallucinationNLIModel.from_pretrained(Config.MODEL_NAME).to(Config.DEVICE)

# load state_dict đã train
ckpt_path = "/kaggle/input/nli-crossencoder/best_nli_sota_model.pth"
state_dict = torch.load(ckpt_path, map_location=Config.DEVICE)
model.load_state_dict(state_dict)
model.eval()

# ===============================
# 4. Inference
# ===============================
test_preds = []
with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(Config.DEVICE) for k, v in batch.items()}
        logits = model(**batch).logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        test_preds.extend(preds)

# map lại nhãn
id2label = {0: "no", 1: "extrinsic", 2: "intrinsic"}
final_labels = [id2label[p] for p in test_preds]

# ===============================
# 5. Xuất submission
# ===============================
submission_df = pd.DataFrame({
    "id": test_df["id"],
    "predict_label": final_labels
})
submission_df.to_csv("submission_nli_sota799_p.csv", index=False)
print(submission_df.head())



                                     id predict_label
0  ef35e7a1-766f-4455-b349-09084a1f56ba            no
1  85aac4aa-e53c-4d01-bdcf-e8b27a2cd9bc     intrinsic
2  cd056e1b-51f6-4adf-939c-e742645437bf     intrinsic
3  6cc80aa4-44db-4366-8cb6-16453aa8ecbf     intrinsic
4  948849cb-0c83-4dc1-b5ae-aaa585a8d528     intrinsic


### 1.2.2 Load 2 model từ Hugging Face

In [11]:
!pip install huggingface_hub


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
from huggingface_hub import hf_hub_download

# tải file best_stable_nli_model.pth từ repo
ckpt_path = hf_hub_download(
    repo_id="Nguyenhhh/vihallu_model_mdeberta_nli_hard30",  # tên repo trên HF
    #Nguyenhhh/vihallu_model_mdeberta_nli_hard01
    filename="best_stable_nli_model.pth"                   # tên file trong repo
)
print("Checkpoint path:", ckpt_path)

# load vào model
state_dict = torch.load(ckpt_path, map_location=Config.DEVICE)
model.load_state_dict(state_dict)


best_stable_nli_model.pth:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Checkpoint path: /root/.cache/huggingface/hub/models--Nguyenhhh--vihallu_model_mdeberta_nli_hard30/snapshots/4d807ddb45665e752efac4bca7bea18a458a2366/best_stable_nli_model.pth


<All keys matched successfully>

In [20]:

class Config:
    TEST_PATH = "/kaggle/working/public_processed.csv"
    MODEL_NAME = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"  # backbone
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    MAX_LEN = 1024
    BATCH_SIZE = 16

# ===============================
# 2. Dataset cho inference
# ===============================
tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_NAME)
test_df = pd.read_csv(Config.TEST_PATH)

class NLIDataset(Dataset):
    def __init__(self, df):
        self.premises = df['premise'].astype(str).tolist()
        self.hypotheses = df['hypothesis'].astype(str).tolist()
    def __len__(self): return len(self.premises)
    def __getitem__(self, idx):
        item = tokenizer(
            self.premises[idx], self.hypotheses[idx],
            padding="max_length", truncation=True,
            max_length=Config.MAX_LEN, return_tensors="pt"
        )
        return {k: v.squeeze(0) for k, v in item.items()}

test_loader = DataLoader(NLIDataset(test_df), batch_size=Config.BATCH_SIZE, shuffle=False)



test_preds = []
with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(Config.DEVICE) for k, v in batch.items()}
        logits = model(**batch).logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        test_preds.extend(preds)

id2label = {0: "no", 1: "extrinsic", 2: "intrinsic"}
final_labels = [id2label[p] for p in test_preds]

# ===============================
# 5. Xuất submission
# ===============================
submission_df = pd.DataFrame({
    "id": test_df["id"],
    "predict_label": final_labels
})
submission_df.to_csv("submission_nli_sota801.csv", index=False)
print(submission_df.head())

                                     id predict_label
0  b709059b-b3b6-4ac2-bb88-2c794e2cc219     extrinsic
1  7dc35ef5-c4b7-4538-ab90-627b9cbd896e     extrinsic
2  cfdfa010-f61c-4845-91c9-23f79be2b88b     extrinsic
3  31b33c97-2f59-4e72-8707-f47de204d7f9     intrinsic
4  a2c83a00-e8b7-4236-86ce-5e0104df074a     intrinsic


In [21]:
df1 = submission_df
df2 = pd.read_csv("/kaggle/input/801pbtest/submit.csv")

num_diff, diff_rows = compare_predict_label(df1, df2)

print(diff_rows.head())  # in vài dòng đầu khác nhau

🔎 Số giá trị khác nhau ở cột `predict_label`: 64
                                      df1                \
                                       id predict_label   
32   ae7ea16b-f316-4c1d-b0d0-7f2a1465cdae     extrinsic   
47   65473e52-7738-405c-a3b3-fbc5896e03bd     intrinsic   
50   9c463d20-ed7f-4b76-baa5-12c214987315     intrinsic   
60   6dd5e6db-7bc2-4681-b25c-fbfd4c299ef3            no   
145  450408cf-89da-42ac-a4e0-c22b303e8d03     extrinsic   

                                      df2                
                                       id predict_label  
32   ae7ea16b-f316-4c1d-b0d0-7f2a1465cdae     intrinsic  
47   65473e52-7738-405c-a3b3-fbc5896e03bd            no  
50   9c463d20-ed7f-4b76-baa5-12c214987315            no  
60   6dd5e6db-7bc2-4681-b25c-fbfd4c299ef3     extrinsic  
145  450408cf-89da-42ac-a4e0-c22b303e8d03            no  


#### Model 807

In [17]:
from huggingface_hub import hf_hub_download

# tải file best_stable_nli_model.pth từ repo
ckpt_path = hf_hub_download(
    repo_id="Nguyenhhh/vihallu_model_mdeberta_nli_hard01",  # tên repo trên HF
    #Nguyenhhh/vihallu_model_mdeberta_nli_hard01
    filename="best_stable_nli_model.pth"                   # tên file trong repo
)
print("Checkpoint path:", ckpt_path)

# load vào model
state_dict = torch.load(ckpt_path, map_location=Config.DEVICE)
model.load_state_dict(state_dict)

best_stable_nli_model.pth:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Checkpoint path: /root/.cache/huggingface/hub/models--Nguyenhhh--vihallu_model_mdeberta_nli_hard01/snapshots/074274b0747d336e64598df9c34f043d3c255bc5/best_stable_nli_model.pth


<All keys matched successfully>

In [29]:

class Config:
    TEST_PATH = "/kaggle/working/private_processed.csv"
    MODEL_NAME = "MoritzLaurer/mDeBERTa-v3-base-mnli-xnli"  # backbone
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    MAX_LEN = 1024
    BATCH_SIZE = 16

# ===============================
# 2. Dataset cho inference
# ===============================
tokenizer = AutoTokenizer.from_pretrained(Config.MODEL_NAME)
test_df = pd.read_csv(Config.TEST_PATH)

class NLIDataset(Dataset):
    def __init__(self, df):
        self.premises = df['premise'].astype(str).tolist()
        self.hypotheses = df['hypothesis'].astype(str).tolist()
    def __len__(self): return len(self.premises)
    def __getitem__(self, idx):
        item = tokenizer(
            self.premises[idx], self.hypotheses[idx],
            padding="max_length", truncation=True,
            max_length=Config.MAX_LEN, return_tensors="pt"
        )
        return {k: v.squeeze(0) for k, v in item.items()}

test_loader = DataLoader(NLIDataset(test_df), batch_size=Config.BATCH_SIZE, shuffle=False)



test_preds = []
with torch.no_grad():
    for batch in test_loader:
        batch = {k: v.to(Config.DEVICE) for k, v in batch.items()}
        logits = model(**batch).logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        test_preds.extend(preds)

id2label = {0: "no", 1: "extrinsic", 2: "intrinsic"}
final_labels = [id2label[p] for p in test_preds]

# ===============================
# 5. Xuất submission
# ===============================
submission_df = pd.DataFrame({
    "id": test_df["id"],
    "predict_label": final_labels
})
submission_df.to_csv("submission_nli_sota807.csv", index=False)
print(submission_df.head())

                                     id predict_label
0  ef35e7a1-766f-4455-b349-09084a1f56ba            no
1  85aac4aa-e53c-4d01-bdcf-e8b27a2cd9bc     intrinsic
2  cd056e1b-51f6-4adf-939c-e742645437bf     intrinsic
3  6cc80aa4-44db-4366-8cb6-16453aa8ecbf     intrinsic
4  948849cb-0c83-4dc1-b5ae-aaa585a8d528     intrinsic


# Hàm đổi 

In [18]:
import pandas as pd

def apply_rules_and_update(path_df1, path_df2, path_df3,
                           output_df3="AIMon_updated.csv",
                           output_df2="submit_df2_updated2.csv"):
    """
    Áp dụng rule-based update cho 3 file CSV:
      - df1: submit.csv 1
      - df2: submit.csv 2
      - df3: AIMon.csv (có cột results + predict_label)
    """

    # Đọc dữ liệu (ép predict_label thành string để tránh lỗi dtype)
    df1 = pd.read_csv(path_df1)
    df2 = pd.read_csv(path_df2)
    df3 = pd.read_csv(path_df3, dtype={"predict_label": str})

    # Copy df2 để update
    df4 = df2.copy()

    # So sánh trực tiếp theo index
    diff_mask = df1["predict_label"] != df2["predict_label"]

    # Lấy ra các dòng khác nhau
    diff_rows = pd.concat([df1[diff_mask], df2[diff_mask]], axis=1, keys=["df1", "df2"])
    diff_rows.columns = [f"{a}_{b}" for a, b in diff_rows.columns]

    # Gộp thêm từ df3
    diff_rows = diff_rows.merge(
        df3[["predict_label", "results"]],
        left_index=True, right_index=True, how="left"
    )

    print("🔎 Số giá trị khác nhau ở cột predict_label:", diff_mask.sum())

    # ===============================
    # Rule 1 (theo nhãn df1)
    # ===============================
    mask_update_1 = (
        (diff_rows["df2_predict_label"] == "no") &
        (diff_rows["results"] > 0.8) 
      
    )
    update_idx_1 = diff_rows.index[mask_update_1]

    if len(update_idx_1) > 0:
        print("\n📌 Rule 1 (theo df1) - Các dòng cập nhật:")
        print(diff_rows.loc[update_idx_1, ["df1_predict_label", "df2_predict_label", "results"]])

    # cập nhật theo nhãn df1
        df3.loc[update_idx_1, "predict_label"] = diff_rows.loc[update_idx_1, "df1_predict_label"].values
        df4.loc[update_idx_1, "predict_label"] = diff_rows.loc[update_idx_1, "df1_predict_label"].values

    # in chi tiết theo từng nhãn df1
        updated_counts = diff_rows.loc[update_idx_1, "df1_predict_label"].value_counts().to_dict()
        print(f"✅ Rule 1: Đã cập nhật {len(update_idx_1)} dòng theo df1 → {updated_counts}")


    # ===============================
    # Rule 2 (giữ nguyên logic cũ)
    # ===============================
    mask_update_2 = (
        (diff_rows["df2_predict_label"] == "intrinsic") &
        (diff_rows["results"] < 0.1)
    )
    update_idx_2 = diff_rows.index[mask_update_2]

    if len(update_idx_2) > 0:
        print("\n📌 Rule 2 - Các dòng cập nhật:")
        print(diff_rows.loc[update_idx_2, ["df2_predict_label", "results"]])

        df3.loc[update_idx_2, "predict_label"] = "no"
        df4.loc[update_idx_2, "predict_label"] = "no"

        print(f"✅ Rule 2: Đã cập nhật {len(update_idx_2)} dòng từ 'intrinsic' → 'no'.")

    # ===============================
    # Phân tích thống kê
    # ===============================
    print("\n📊 Phân tích trong diff_rows:")
    stats = (
        diff_rows.groupby("df2_predict_label")["results"]
        .agg(["count", "min", "max", "median"])
    )
    print(stats)

    # Xuất file
    df3.to_csv(output_df3, index=False)
    df4.to_csv(output_df2, index=False)

    print("\n🎉 Hoàn tất cập nhật theo 2 rule (chỉ áp dụng trong diff_rows)!")

    return df3, df4, stats


In [20]:
df3_updated, df2_updated, diff_rows = apply_rules_and_update(
    "/kaggle/working/submission_majority.csv",
    "/kaggle/input/800privatetest/submit.csv",
    "/kaggle/input/private-ai-mon/result.csv"
)


🔎 Số giá trị khác nhau ở cột predict_label: 82

📌 Rule 1 (theo df1) - Các dòng cập nhật:
                                        id df1_predict_label  \
269   9de2f9ed-20ae-41d9-9792-3ce57266d4a1         extrinsic   
462   26baafeb-b248-41f9-af32-318b363c9589         extrinsic   
501   70d2c7d0-5e2c-462f-9893-2159517060f1         extrinsic   
723   b6085276-44f0-4fa2-ab9d-291d6b56da7f         intrinsic   
905   1825d5cc-b67c-4664-8543-467f558bb749         intrinsic   
1231  e3c0e21b-39b4-4c1b-94c4-fb531e68581e         extrinsic   
1357  1aabd56e-d7c9-4868-b5e3-189553661a9d         extrinsic   
1454  12e31de2-a9d8-496d-a1ec-dd5d48fdc84a         extrinsic   
1648  b9c3a485-f61f-47b9-9e5e-9a97bcece40f         extrinsic   

     df2_predict_label   results  
269                 no  0.820312  
462                 no  0.863281  
501                 no  0.929688  
723                 no  0.972656  
905                 no  0.925781  
1231                no  0.964844  
1357                no  0